# **End-to-end weakly-supervised semantic alignment**

**Authors: Ignacio Rocco1,2, Relja Arandjelovic´3, Josef Sivic1,2,4**

**1 DI ENS / 2 Inria / 3 DeepMind / 4 CIIRC, CTU in Prague**

**[Official Github Code](https://github.com/ignacio-rocco/weakalign)** / **[Project Page](https://www.di.ens.fr/willow/research/weakalign/)** / **[Pdf](https://arxiv.org/abs/1712.06861)**

---

**Edited By Su Hyung Choi (Key Summary & Code Practice)**

If you have any issues on this scripts, please PR to the repository below.

**[Github: @JonyChoi - Computer Vision Paper Reviews](https://github.com/jonychoi/Computer-Vision-Paper-Reviews)**

Edited March 21 2022

---

## **Abstract**

***"We tackle the task of semantic alignment where the goal
is to compute dense semantic correspondence aligning two
images depicting objects of the same category."***

#### **Contribution**

---

***First***, *we develop a convolutional neural network architecture for semantic alignment
that is trainable in an end-to-end manner from weak imagelevel supervision in the form of matching image pairs.*

> ***Semi-Local Constraints*를 이용하여 애매모한 Feature를 매칭시키는 클래식한 아이디어에서 영감을 얻은바, Global Gemoetric Model의 필요 없이, 4D Space에서 가능한 모든 neighbourhood consensus 패턴들을 분석해서 spatially matching하는, End-to-End로 학습가능한 CNN Architecture를 개발.**

---

***Second***, *the main component of this architecture is a differentiable soft inlier scoring module, inspired by the RANSAC inlier scoring procedure, that computes the quality of the alignment based on
only geometrically consistent correspondences thereby reducing the effect of background clutter.*

> **Manual annotation, 즉 완전 하드 annotated 방식이 아닌 matching과 non-matchinig image pairs 형태를 갖춘 weak supervision을 통해 효율적 학습**

---

***Third***, *we demonstrate that the proposed approach achieves state-of-the-art
performance on multiple standard benchmarks for semantic
alignment.*

> **여기서는 정성적 평가. Neighbourhood Consensus Network는 Category, Instance-level Matching을 포함한 넓은 matching task에서 PF PASCAL 데이터셋과 InLoc Indoor Visual Localization 벤치마크에서 소타달성.**

---

## **Introduction**

![](https://www.di.ens.fr/willow/research/weakalign/images/teaser.jpg)

> **기존의 Visual Correspondence는 *Viewpoint*의 변화나, *illumination* 등 다양한 variation에 꽤 괜찮은 결과를 낳았으나, 여전히 hand crafted 모델에 비해서 trainable한 모델들이 근소하게 좋은 성능을 보이고 있음. (OD나 Classification과 다르게)**

*"One of the reasons for this plateauing performance could be the currently dominant approach for
finding image correspondence based on matching individual image features. While we have now better
local patch descriptors, the matching is still performed by variants of the nearest neighbour assignment
in a feature space followed by separate disambiguation stages based on geometric constraints."*

> **애매모하고 variate한 feature들 분리하고, invariate한 부분들 위주로 가장 가까운 neighbour을 갖다가 assign해버리는 individual image feature 기반의 image correspondence 가 dominant한 approach.**

*"This
approach has, however, fundamental limitations. Imagine a scene with textureless regions or repetitive
patterns, such as a corridor with almost textureless walls and only few distinguishing features. A
small patch of an image, depicting a repetitive pattern or a textureless area, is indistinguishable from other portions of the image depicting the same repetitive or textureless pattern. Such matches will be
either discarded [23] or incorrect. As a result, matching individual patch descriptors will often fail in
such challenging situations."*

> **그러나 이건 근본적인 문제점. Textureless한 region 또는 반복되는 패턴의 회랑 같은 것들은 조금의 분류가능한 feature들을 가지고 있음. 때문에 이러한 Individual한 patch descriptors는 이러한 challenging한 상황에서 실패**


*"In this work we take a different direction and develop a trainable neural network architecture that
disambiguates such challenging situations by analyzing local neighbourhood patterns in a full set of
dense correspondences. **The intuition is the following: in order to disambiguate a match on a repetitive
pattern, it is necessary to analyze a larger context of the scene that contains a unique non-repetitive
feature.** The information from this unique match can then be propagated to the neighbouring uncertain
matches. **In other words, the certain unique matches will support the close-by uncertain ambiguous
matches in the image."***

> **그래서 큰 Context에서 Unique한 것들이 가까운 애매모한 것들을 Matching하는 걸 support함으로써 uncertain한 것들을 처리하자!! + trainable CNN은 당연하고**
